In [10]:
# Ensure you have the following installed:
# numpy
# os
# compas
# compas_fd
# warnings
# traceback
# pickle

# Install by running:
# !pip install -r requirements.txt

In [11]:
import numpy as np
import pickle
import pandas as pd

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import save_params, load_params, printer_start, load_tool, unload_tool, tool_change, take_photo, play_sound, printer_stop
# from src.additional_functions import *
from src.network import Network_custom, replace_brackets
import os
import plotly.io as pio
pio.renderers.default = 'browser'
BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"

### Step 1: Define the network

In [12]:
def Rot(theta):
    """
    Rotation matrix around the z-axis
    :param theta: angle in radians
    :return: rotation matrix
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

# Note that the coordinates of the vertices will change after equilibrium is reached.
model_name = "validation_unit"
L_scale = 10.5
vertices    = np.array([[-10, 0, 0], [-5, 0,0], [0, 0, 0], [5, 0, 0], [10, 0, 0], [0,-10,0], [0,-5,0], [0,5,0], [0,10,0]])*L_scale  # The coordinates of the vertices in mm
vertices    = vertices@Rot(np.pi/4)
edges       = [[0, 1], [1, 2], [2, 3], [3, 4], [5, 6], [6, 2], [2, 7], [7, 8], [1, 6], [6, 3], [3, 7], [7,1]]                  # The edges of the network.
paths       = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]   # 3D printable paths. Ensure an edge is not flipped.
directions  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]          # Flip the curvature of an edge by setting the direction to -1.
center_cross_path = [1, 2, 5, 6]
connector_path = [0, 3, 4, 7]
q_scalar = 1.4
sum_q0 = 0.02*q_scalar
sum_q1 = 0.02*q_scalar
sum_q2 = 0.01*q_scalar
offset = 1/32
ratio_0 = np.array([3/8-offset, 1/8, 1/8, 3/8+offset])
ratio_1 = np.array([3/8, 1/8-offset, 1/8+offset, 3/8])
ratio_2 = np.array([2/8-offset, 2/8+offset, 2/8, 2/8])

q = np.zeros(len(edges))
q[paths[0]] = sum_q0*ratio_0
q[paths[1]] = sum_q1*ratio_1
q[paths[2]] = sum_q2*ratio_2

fixed       = [0, 4, 5, 8]
# Create the network
net = Network_custom.from_fd(vertices, edges, q, fixed, paths = paths, dir = directions)
net.net_plot(color=True, elables = False, vlabels = True)
np.max(net.f)/0.078294515

7.321273822984715

Constant settings

In [22]:
7.3/0.078294515

93.23769359833189

In [13]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm_new3.json')
alpha_loop = np.deg2rad(30) # The angle of the loop
L_loop  = 9                # The length of the loop
n_points = 30               # The number of points in the loop
net.n_split = n_points

start_gcode     = open(r'DATA/start_gcode_XL.gcode', 'r').read()
end_gcode       = open(r'DATA/end_gcode.gcode', 'r').read()

temperature_settings = {'first_layer_bed_temperature': 60, 'first_layer_temperature':220, 'K-factor': 0.20, 'idle_temperature': printing_params["T_nozzle_standby"]}
comment = ''

bed_width = 230
bed_height = 210

start_loop_bools = [True, True, False]
end_loop_bools = [True, True, False]

running_start_bools = [True, True, True]
running_end_bools = [True, True, False]
L_running = 6

file_path = os.path.join(BYU_UW_root, 'Tensile Testing', 'Avg_Stress_Strain_Overture_TPU.csv')
stress_data, strain_data = net.load_stress_strain_curve(file_path, A_scale = 1.0)

TPU_nl = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A': 0.078294515, 'name': 'TPU Overture'} # TPU Overture non-conductive

A = [TPU_nl['A']]*len(net.edges)

# Purge line coors
gen = G_code_generator(printing_params)
point0, point1 = [20,15,printing_params['layer_height']-printing_params['first_layer_offset']], [150,15,printing_params['layer_height']-printing_params['first_layer_offset']]
point2, point3 = [150,10,printing_params['layer_height']-printing_params['first_layer_offset']], [20,10,printing_params['layer_height']-printing_params['first_layer_offset']]

def make_purge_line(printing_params, gen, point0, point1, ef_extra = 1.0, comment=''):
    """"
    ""Make a purge line at the start of the print."
    """
    g_code = '\n'
    g_code += gen.move_to_point(point0[0:2], point0[2] + printing_params['nozzle_lift'], comment='Move to start point')
    g_code += gen.move_to_point(point0[0:2], point0[2], comment='Lower Nozzle')
    g_code += gen.unretract()
    g_code += gen._print_line(
            point0=point0,
            point1=point1,
            move_to_start=False, # move to start point without extruding
            extrude_factor=printing_params['extrude_factor']*ef_extra,
            comment=comment)
    g_code += gen.retract()
    g_code += gen.wipe(2 * np.pi)
    g_code += '\n'
    return g_code

def add_net_to_gcode(printing_params, gen, net):
    g_code = '\n'
    for path_i, cor_list in enumerate(net.paths_xyz):
        g_code += '\n;Path (' + str(path_i) + ' ' + str(path_i) + ')\n'
        cor_list   = np.array(cor_list)
        # Move the coordinates to the center of the bed and add the layer height
        cor_list[:,0] += bed_width/2 -14
        cor_list[:,1] += bed_height/2 
        cor_list[:,2] += (printing_params['layer_height'] - printing_params['first_layer_offset'])
        g_code += '\n'
        # Move the coordinates to the start point
        g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2] + printing_params['nozzle_lift'], comment='Move to start point')
        # Lower the nozzle
        g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2], comment='Lower Nozzle')
        # Unretract the filament
        g_code += gen.unretract()
        # Print the path
        for point0, point1 in zip(cor_list[:-1], cor_list[1:]):
                g_code += gen._print_line(
                point0=point0,
                point1=point1,
                move_to_start=False, # move to start point without extruding
                extrude_factor = printing_params['extrude_factor'],
                comment=comment)
        # Retract the filament
        g_code += gen.retract()
        # Wipe the nozzle
        g_code += gen.wipe_from_last_points(g_code)    
        # Raise the nozzle
        g_code += gen.move_to_point(point1[0:2], point1[2] + printing_params['nozzle_lift'], speed_factor=0.5, comment='Raise Nozzle')
    return g_code

In [14]:
# import matplotlib.pyplot as plt

# file_path = os.path.join(BYU_UW_root, 'Tensile Testing', 'Avg_Stress_Strain_Overture_TPU.csv')
# stress_data_1mm2, strain_data_1mm2 = net.load_stress_strain_curve(file_path, A_scale = 1.0)

# file_path = os.path.join(BYU_UW_root, 'Tensile Testing', 'Avg_Stress_Strain_Overture_TPU.csv')
# stress_data_0, strain_data_0 = net.load_stress_strain_curve(file_path, A_scale = 1.0)

# fix, ax = plt.subplots(1, 1, figsize=(10, 10))
# ax.set_title('Stress Strain Curve')
# ax.set_xlabel('Strain')
# ax.set_ylabel('Stress (MPa)')
# ax.plot(strain_data_1mm2, stress_data_1mm2, label='Stress Strain Curve 1mm2')
# ax.plot(strain_data_0, stress_data_0, label='Stress Strain Curve 0')
# ax.legend()

Generate Specimen

In [15]:
reference_point = [0,0,0] 
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .01, "correction_scalar": 1., "tol": 1e-6})
net.optimize_vertices()
net.scale_vertices(reference_point, net.l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
R, th = net.arc_param()
xyz = net.arc_points(n = n_points)
net.auto_flip_curves(n = n_points)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
net.flip_curves(n = n_points)                   # Flip all the edges.
net.flip_curves([2, 5, 8], n = n_points)
net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 
net.net_plot(plot_type = 'arcs')
net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)
net.add_running_start(running_start_bools, running_end_bools, L_running, n_points)
file_name = model_name + f'_0'
net.save_network(os.path.join(BYU_UW_root, 'networks', file_name + '.pkl'))
# print(f"q_scalar: {q_scalar}, l_scalar: {l_scalar}, max stress: {np.max(net.f)/TPU_nl['A']}")

g_code = replace_brackets(start_gcode, temperature_settings)
g_code += make_purge_line(printing_params, gen, point0, point1, comment='Purge line', ef_extra=2.5)
g_code += make_purge_line(printing_params, gen, point2, point3, comment='Purge line', ef_extra=1.05)
g_code += add_net_to_gcode(printing_params, gen, net)

g_code += end_gcode
with open(os.path.join('DATA', 'generated_gcodes', file_name + '.gcode'), "w") as g_code_file:
    g_code_file.write(g_code)
net.net_plot(color=True, elables = False, vlabels = True, plot_type = "arcs")

Iteration 0: Current error = 272.573939211011
Iteration 100: Current error = 119.12784505408679
Iteration 200: Current error = 57.54004856448809
Iteration 300: Current error = 28.603858714932514
Iteration 400: Current error = 14.772604081096818
Iteration 500: Current error = 8.082294310092605
Iteration 600: Current error = 4.81963485705824
Iteration 700: Current error = 3.2195253503338246
Iteration 800: Current error = 2.4316978773297113
Iteration 900: Current error = 2.042791367211962
Final error: 1.8519229196596252


Generate 6 specimens only changing the tension

In [16]:
q0 = q.copy()
q_scalars = [0.35, 0.6, 0.8,1.0, 1.15, 1.25]
for q_scalar in q_scalars:
    file_name = model_name + f'_fstudy_{q_scalar}'
    q_new = q0*q_scalar
    # net.update_shape(q_new)
    net = Network_custom.from_fd(vertices, edges, q_new, fixed, paths = paths, dir = directions)
    l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
    net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .01, "correction_scalar": 1., "tol": 1e-6})
    net.optimize_vertices()
    net.scale_vertices(reference_point, net.l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
    R, th = net.arc_param()
    xyz = net.arc_points(n = n_points)
    net.auto_flip_curves(n = n_points)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
    net.flip_curves(n = n_points)                   # Flip all the edges.
    net.flip_curves([2, 5, 8], n = n_points)
    net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 

    net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)
    net.add_running_start(running_start_bools, running_end_bools, L_running, n_points)
    
    net.save_network(os.path.join(BYU_UW_root, 'networks', file_name + '.pkl'))
    print(f"max stress: {np.max(net.f)/TPU_nl['A']}")

    g_code = replace_brackets(start_gcode, temperature_settings)
    g_code += make_purge_line(printing_params, gen, point0, point1, comment='Purge line', ef_extra=2.5)
    g_code += make_purge_line(printing_params, gen, point2, point3, comment='Purge line', ef_extra=1.05)
    g_code += add_net_to_gcode(printing_params, gen, net)

    g_code += end_gcode
    with open(os.path.join('DATA', 'generated_gcodes', file_name + '.gcode'), "w") as g_code_file:
        g_code_file.write(g_code)
    net.net_plot(color=True, elables = False, vlabels = True, plot_type = "arcs")
    tikz_text = net.generate_tikz_figure(paths = net.paths_xyz)
    with open(os.path.join('DATA', 'tikz_images', file_name + '.tex'), "w") as tikz_file:
        tikz_file.write(tikz_text)
# tikz_text

Iteration 0: Current error = 10.175656330124905
Iteration 100: Current error = 5.0013974640068195
Iteration 200: Current error = 2.5110351815059313
Iteration 300: Current error = 1.286514551064735
Iteration 400: Current error = 0.6822058727033593
Iteration 500: Current error = 0.38329325810328463
Iteration 600: Current error = 0.23518262578205087
Iteration 700: Current error = 0.1617101607878121
Iteration 800: Current error = 0.1252280914947302
Iteration 900: Current error = 0.10710818307161607
Final error: 0.09816497407086341
max stress: 2.562445838044651
Iteration 0: Current error = 44.97608560689359
Iteration 100: Current error = 21.470774728153533
Iteration 200: Current error = 10.664444474340932
Iteration 300: Current error = 5.414049739619553
Iteration 400: Current error = 2.8443722590607585
Iteration 500: Current error = 1.580683366874842
Iteration 600: Current error = 0.9572122402699034
Iteration 700: Current error = 0.6488704826801233
Iteration 800: Current error = 0.496129094

In [17]:
# net.update_shape(q_new)
q0 = q.copy()*0.8
net = Network_custom.from_fd(vertices, edges, q0, fixed, paths = paths, dir = directions)
l1_initial = net.l1.copy()
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .01, "correction_scalar": 1., "tol": 1e-6})
# net.initialize_shape_optimizer(function_type = 'standard',  method = 'L-BFGS-B',options ={"maxiter": 10000, "maxfun": 1500000, "maxls": 5000})
net.optimize_vertices()
reference_point = [0,0,0]                         # The network will be scaler relative to this point

l_scalar_vec = np.linspace(.65, net.l_scalar, 6)
for l_scalar in l_scalar_vec:

    net.scale_vertices(reference_point, l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
    print(np.min(net.l1_scaled/net.l0))

    R, th = net.arc_param()
    xyz = net.arc_points(n = n_points)
    net.auto_flip_curves(n = n_points)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
    net.flip_curves(n = n_points)                   # Flip all the edges.
    net.flip_curves([8, 10], n = n_points)
    net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 
    net.net_plot(color=True, plot_type='arcs', elables=True, vlabels = True)
    net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)
    net.add_running_start(running_start_bools, running_end_bools, L_running, n_points)
    file_name = model_name + f'_ls{l_scalar:.2f}'
    net.save_network(os.path.join(BYU_UW_root, 'networks', file_name + '.pkl'))
    # print(f"q_scalar: {q_scalar}, l_scalar: {l_scalar}, max stress: {np.max(net.f)/TPU_nl['A']}")

    g_code = replace_brackets(start_gcode, temperature_settings)
    g_code += make_purge_line(printing_params, gen, point0, point1, comment='Purge line', ef_extra=2.5)
    g_code += make_purge_line(printing_params, gen, point2, point3, comment='Purge line', ef_extra=1.0)
    g_code += add_net_to_gcode(printing_params, gen, net)

    g_code += end_gcode
    with open(os.path.join('DATA', 'generated_gcodes', file_name + '.gcode'), "w") as g_code_file:
        g_code_file.write(g_code)
    
    tikz_text = net.generate_tikz_figure(paths = net.paths_xyz)
    with open(os.path.join('DATA', 'tikz_images', file_name + '.tex'), "w") as tikz_file:
        tikz_file.write(tikz_text)


Iteration 0: Current error = 115.65190922748374
Iteration 100: Current error = 53.230327481536236
Iteration 200: Current error = 26.13450994299013
Iteration 300: Current error = 13.150265406923328
Iteration 400: Current error = 6.856180094774227
Iteration 500: Current error = 3.7819943221672103
Iteration 600: Current error = 2.2723465219947916
Iteration 700: Current error = 1.5283009509700347
Iteration 800: Current error = 1.1606061313462117
Iteration 900: Current error = 0.978643083673215
Final error: 0.8891280009977779
0.6447931564564436
0.7132409217387713
0.7816886870210991
0.8501364523034266
0.9185842175857544
0.987031982868082


In [18]:
q0 = q.copy()
q_scalar = 0.8
for q_scalar in q_scalars:
    file_name = model_name + f'_fstudy_{q_scalar}'
    q_new = q0*q_scalar
    # net.update_shape(q_new)
    net = Network_custom.from_fd(vertices, edges, q_new, fixed, paths = paths, dir = directions)
    l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
    net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .01, "correction_scalar": 1., "tol": 1e-6})
    net.optimize_vertices()
    net.scale_vertices(reference_point, net.l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
    R, th = net.arc_param()
    xyz = net.arc_points(n = n_points)
    net.auto_flip_curves(n = n_points)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
    net.flip_curves(n = n_points)                   # Flip all the edges.
    net.flip_curves([2, 5, 8], n = n_points)
    net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 

    net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)
    net.add_running_start(running_start_bools, running_end_bools, L_running, n_points)
    
    net.save_network(os.path.join(BYU_UW_root, 'networks', file_name + '.pkl'))
    print(f"max stress: {np.max(net.f)/TPU_nl['A']}")

    g_code = replace_brackets(start_gcode, temperature_settings)
    g_code += make_purge_line(printing_params, gen, point0, point1, comment='Purge line', ef_extra=2.5)
    g_code += make_purge_line(printing_params, gen, point2, point3, comment='Purge line', ef_extra=1.05)
    g_code += add_net_to_gcode(printing_params, gen, net)

    g_code += end_gcode
    with open(os.path.join('DATA', 'generated_gcodes', file_name + '.gcode'), "w") as g_code_file:
        g_code_file.write(g_code)
    net.net_plot(color=True, elables = False, vlabels = True, plot_type = "arcs")
    tikz_text = net.generate_tikz_figure(paths = net.paths_xyz)
    with open(os.path.join('DATA', 'tikz_images', file_name + '.tex'), "w") as tikz_file:
        tikz_file.write(tikz_text)
    

Iteration 0: Current error = 10.175699884989033
Iteration 100: Current error = 5.001494696513542
Iteration 200: Current error = 2.511066451831598
Iteration 300: Current error = 1.2865411603023007
Iteration 400: Current error = 0.6822224249267995
Iteration 500: Current error = 0.38331141308378336
Iteration 600: Current error = 0.23518729586012996
Iteration 700: Current error = 0.1617154233892591
Iteration 800: Current error = 0.12523972683919887
Iteration 900: Current error = 0.10711689391058309
Final error: 0.09816756846941976
max stress: 2.562445838044651
Iteration 0: Current error = 44.9761211902488
Iteration 100: Current error = 21.4706126953398
Iteration 200: Current error = 10.66440346363738
Iteration 300: Current error = 5.4140710619585
Iteration 400: Current error = 2.844389549361804
Iteration 500: Current error = 1.5807676551034147
Iteration 600: Current error = 0.9572745504845579
Iteration 700: Current error = 0.6488933588825252
Iteration 800: Current error = 0.496136562721500

In [19]:
minx =10000
miny=10000
maxx=0
maxy=0
for path_xyz in net.paths_xyz:
    minxl = np.min(path_xyz[:,0])
    maxxl = np.max(path_xyz[:,0])
    minyl = np.min(path_xyz[:,1])
    maxyl = np.max(path_xyz[:,1])
    minx = np.min([minxl, minx])
    maxx = np.max([maxxl, maxx])
    miny = np.min([minyl, miny])
    maxy = np.max([maxyl, maxy])


minx += bed_width/2 -14
miny += bed_height/2 
maxx += bed_width/2 -14
maxy += bed_height/2 
minx ,miny , maxx , maxy

(29.504969873693042, 33.34832430367693, 171.6952424282797, 176.82398320419242)

In [20]:
pio.renderers.default = 'browser'
# Note that the coordinates of the vertices will change after equilibrium is reached.
model_name = "validation_unit_s"
L_scale = 3.5
vertices    = np.array([[-10, 0, 0], [-5, 0,0], [0, 0, 0], [5, 0, 0], [10, 0, 0], [0,-10,0], [0,-5,0], [0,5,0], [0,10,0]])*L_scale  # The coordinates of the vertices in mm
vertices    = vertices@Rot(np.pi/4)
edges       = [[0, 1], [1, 2], [2, 3], [3, 4], [5, 6], [6, 2], [2, 7], [7, 8], [1, 6], [6, 3], [3, 7], [7,1]]                  # The edges of the network.
paths       = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]   # 3D printable paths. Ensure an edge is not flipped.
directions  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]          # Flip the curvature of an edge by setting the direction to -1.
center_cross_path = [1, 2, 5, 6]
connector_path = [0, 3, 4, 7]
q_scalar = 1.4
sum_q0 = 0.02*q_scalar
sum_q1 = 0.02*q_scalar
sum_q2 = 0.01*q_scalar
offset = 1/32
ratio_0 = np.array([3/8-offset, 1/8, 1/8, 3/8+offset])
ratio_1 = np.array([3/8, 1/8-offset, 1/8+offset, 3/8])
ratio_2 = np.array([2/8-offset, 2/8+offset, 2/8, 2/8])

q = np.zeros(len(edges))
q[paths[0]] = sum_q0*ratio_0
q[paths[1]] = sum_q1*ratio_1
q[paths[2]] = sum_q2*ratio_2

# scalar = 7
q[connector_path]       *= 2.5
q[center_cross_path]    *= 2.5
q[paths[2]]             *= 2.5

fixed       = [0, 4, 5, 8]
# Create the network
net = Network_custom.from_fd(vertices, edges, q, fixed, paths = paths, dir = directions)
net.net_plot(color=True, elables = False, vlabels = False)
np.max(net.f)/0.078294515


6.101061519153932

In [21]:
# net.update_shape(q_new)
running_start_bools = [True, True, False]
running_end_bools = [True, True, False]

q0 = q.copy()
net = Network_custom.from_fd(vertices, edges, q0, fixed, paths = paths, dir = directions)
l1_initial = net.l1.copy()
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .01, "correction_scalar": 1., "tol": 1e-6})
# net.initialize_shape_optimizer(function_type = 'standard',  method = 'L-BFGS-B',options ={"maxiter": 10000, "maxfun": 1500000, "maxls": 5000})
net.optimize_vertices()
reference_point = [0,0,0]                         # The network will be scaler relative to this point

l_scalar_vec = np.linspace(.65, net.l_scalar, 6)
for l_scalar in l_scalar_vec:

    net.scale_vertices(reference_point, l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
    print(np.min(net.l1_scaled/net.l0))

    R, th = net.arc_param()
    xyz = net.arc_points(n = n_points)
    net.auto_flip_curves(n = n_points)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
    net.flip_curves(n = n_points)                   # Flip all the edges.
    net.flip_curves([8, 10], n = n_points)
    net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 

    net.net_plot(color=True, plot_type='arcs', elables=True, vlabels = True)
    
    net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)
    net.add_running_start(running_start_bools, running_end_bools, L_running, n_points)
    file_name = model_name + f'_ls{l_scalar:.2f}'
    net.save_network(os.path.join(BYU_UW_root, 'networks', file_name + '.pkl'))
    # print(f"q_scalar: {q_scalar}, l_scalar: {l_scalar}, max stress: {np.max(net.f)/TPU_nl['A']}")

    g_code = replace_brackets(start_gcode, temperature_settings)
    g_code += make_purge_line(printing_params, gen, point0, point1, comment='Purge line', ef_extra=2.5)
    g_code += make_purge_line(printing_params, gen, point2, point3, comment='Purge line', ef_extra=1.0)
    g_code += add_net_to_gcode(printing_params, gen, net)

    g_code += end_gcode
    with open(os.path.join('DATA', 'generated_gcodes', file_name + '.gcode'), "w") as g_code_file:
        g_code_file.write(g_code)
    tikz_text = net.generate_tikz_figure(paths = net.paths_xyz)
    with open(os.path.join('DATA', 'tikz_images', file_name + '.tex'), "w") as tikz_file:
        tikz_file.write(tikz_text)


Iteration 0: Current error = 14.994670575586209
Iteration 100: Current error = 6.84455779010459
Iteration 200: Current error = 3.3519837154047067
Iteration 300: Current error = 1.6832377985907665
Iteration 400: Current error = 0.8760645100354554
Iteration 500: Current error = 0.48238697837703615
Iteration 600: Current error = 0.2892668154493979
Iteration 700: Current error = 0.19415571074614923
Iteration 800: Current error = 0.14719024281679316
Iteration 900: Current error = 0.12395871867489565
Final error: 0.11253568313665561
0.6444544141069666
0.7127975386003631


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 58854)
Traceback (most recent call last):
  File "c:\Users\thijs\anaconda3\Lib\socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\Users\thijs\anaconda3\Lib\socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "c:\Users\thijs\anaconda3\Lib\socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\Users\thijs\anaconda3\Lib\socketserver.py", line 761, in __init__
    self.handle()
  File "c:\Users\thijs\anaconda3\Lib\http\server.py", line 436, in handle
    self.handle_one_request()
  File "c:\Users\thijs\anaconda3\Lib\http\server.py", line 424, in handle_one_request
    method()
  File "c:\Users\thijs\anaconda3\Lib\site-packages\plotly\io\_base_renderers.py", line 698, in do_GET
    self.wfile.write(html[i : i 

0.7811406630937597
0.8494837875871565
0.9178269120805531
0.9861700365739499


In [ ]:
pio.renderers.default = 'browser'
# Note that the coordinates of the vertices will change after equilibrium is reached.
model_name = "validation_unit_xs"
L_scale = 3.5
vertices    = np.array([[-10, 0, 0], [-5, 0,0], [0, 0, 0], [5, 0, 0], [10, 0, 0], [0,-10,0], [0,-5,0], [0,5,0], [0,10,0]])*L_scale  # The coordinates of the vertices in mm
vertices    = vertices@Rot(np.pi/4)
edges       = [[0, 1], [1, 2], [2, 3], [3, 4], [5, 6], [6, 2], [2, 7], [7, 8], [1, 6], [6, 3], [3, 7], [7,1]]                  # The edges of the network.
paths       = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]   # 3D printable paths. Ensure an edge is not flipped.
directions  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]          # Flip the curvature of an edge by setting the direction to -1.
center_cross_path = [1, 2, 5, 6]
connector_path = [0, 3, 4, 7]
q_scalar = 1.4
sum_q0 = 0.02*q_scalar
sum_q1 = 0.02*q_scalar
sum_q2 = 0.01*q_scalar
offset = 1/32
ratio_0 = np.array([3/8-offset, 1/8, 1/8, 3/8+offset])
ratio_1 = np.array([3/8, 1/8-offset, 1/8+offset, 3/8])
ratio_2 = np.array([2/8-offset, 2/8+offset, 2/8, 2/8])

q = np.zeros(len(edges))
q[paths[0]] = sum_q0*ratio_0
q[paths[1]] = sum_q1*ratio_1
q[paths[2]] = sum_q2*ratio_2

# scalar = 7
q[connector_path]       *= 2.5 * 0.65
q[center_cross_path]    *= 2.5 * 4
q[paths[2]]             *= 2.5 * 4

fixed       = [0, 4, 5, 8]
# Create the network
net = Network_custom.from_fd(vertices, edges, q, fixed, paths = paths, dir = directions)
net.net_plot(color=True, elables = False, vlabels = False)
np.max(net.f)/0.078294515

In [ ]:
# net.update_shape(q_new)
q0 = q.copy()
net = Network_custom.from_fd(vertices, edges, q0, fixed, paths = paths, dir = directions)
l1_initial = net.l1.copy()
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .01, "correction_scalar": 1., "tol": 1e-6})
# net.initialize_shape_optimizer(function_type = 'standard',  method = 'L-BFGS-B',options ={"maxiter": 10000, "maxfun": 1500000, "maxls": 5000})
net.optimize_vertices()
reference_point = [0,0,0]                         # The network will be scaler relative to this point

l_scalar_vec = np.linspace(.65, net.l_scalar, 6)
for l_scalar in l_scalar_vec:

    net.scale_vertices(reference_point, l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
    print(np.min(net.l1_scaled/net.l0))

    R, th = net.arc_param()
    xyz = net.arc_points(n = n_points)
    net.auto_flip_curves(n = n_points)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
    net.flip_curves(n = n_points)                   # Flip all the edges.
    net.flip_curves([8, 10], n = n_points)
    net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 

    net.net_plot(color=True, plot_type='arcs', elables=True, vlabels = True)
    
    net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)
    net.add_running_start(running_start_bools, running_end_bools, L_running, n_points)
    file_name = model_name + f'_ls{l_scalar:.2f}'
    net.save_network(os.path.join(BYU_UW_root, 'networks', file_name + '.pkl'))
    # print(f"q_scalar: {q_scalar}, l_scalar: {l_scalar}, max stress: {np.max(net.f)/TPU_nl['A']}")

    g_code = replace_brackets(start_gcode, temperature_settings)
    g_code += make_purge_line(printing_params, gen, point0, point1, comment='Purge line', ef_extra=2.5)
    g_code += make_purge_line(printing_params, gen, point2, point3, comment='Purge line', ef_extra=1.0)
    g_code += add_net_to_gcode(printing_params, gen, net)

    g_code += end_gcode
    with open(os.path.join('DATA', 'generated_gcodes', file_name + '.gcode'), "w") as g_code_file:
        g_code_file.write(g_code)


In [ ]:
net.net_plot(color=True, elables=True, vlabels = True)